In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import TreebankWordTokenizer
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
nltk.download('treebank')
nltk.download('universal_tagset')
import matplotlib.pyplot as mtp  
from sklearn import svm,naive_bayes
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as mtp  
from sklearn import svm,naive_bayes
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
#get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import naive_bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
import json
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
from attributes import domain_dict

import ast #for updating in attributes.py

In [ ]:
pip install language-tool-python

In [ ]:
# importing for testing to remove spelling mistakes

# importing the library  
import language_tool_python  
  
# creating the tool  
spell_correction = language_tool_python.LanguageTool('en-US')  

In [ ]:
pip install graphviz

In [ ]:
from graphviz import Digraph
from PIL import Image

In [ ]:
pip install graphviz Pillow

In [ ]:
# df = pd.read_csv("modifications_dataset.csv")
df = pd.read_csv("learning_dataset.csv")


# Drop any rows with missing values
df = df.dropna()


X = df[['Word', 'POS']].apply(lambda x: ' '.join(x), axis=1)
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_test, y_train, y_test = train_test_split(X, y)
# Convert the words to a matrix of token counts
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
# Convert the test words to a matrix of token counts
X_test_counts = vectorizer.transform(X_test.apply(lambda x: ' '.join(x)))
# Train a Naive Bayes classifier
clf = MultinomialNB().fit(X_train_counts, y_train)
# In[83]:
# Use the classifier to predict the labels of the test set
y_pred = clf.predict(X_test_counts)
# # Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

#geeksforgeeks

text=input("Enter text:")
text = text.lower()
bad_chars = [';', ':', '!', "*","@","(",")",",","be "," be"]
# using replace() to
# remove bad_chars
for i in bad_chars:
	text = text.replace(i, '')
# print(text)
sentences = sent_tokenize(text)
tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()

array_m = []
for sentence in sentences:
    # Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)
    
    # POS tagging using the Averaged Perceptron Tagger
    pos_tags = pos_tag(tokens)
    
    # Lemmatize each word based on its POS tag
    lemmas = []
    for word, tag in pos_tags:
        if tag in ['NN','NNS', 'NNPS', 'POS']:
            lemma = lemmatizer.lemmatize(word, pos='n')
        elif tag in ['RB', 'VBD', 'VBN', 'VBG', 'VB']:
            lemma = lemmatizer.lemmatize(word, pos='v')
        else:
            lemma = word
        lemmas.append((lemma, tag))
    
    array_m.append(lemmas)

# print(array_m)
df_m = pd.DataFrame(array_m)
array = []
for i in range(df_m.shape[0]):
    for j in range(df_m.shape[1]):
        word_pos = df_m.iloc[i, j] # Updated indexing with iloc
        if word_pos is not None:
            word_counts = vectorizer.transform([' '.join(word_pos)])
            label = clf.predict(word_counts)[0]
            if label != "O":
                temp = df_m.iloc[i, j] + (label,) # Updated indexing with iloc
                array.append(temp)
array_list = [tuple(item) for item in array]
# print(array_list)
# code if text has many,more,various make next NN as NNS

output = array_list
flag = False
for i, (word, tag, label) in enumerate(output):
    if word in ['many','more','various','numerous','abundant','multiple','several','plenty']:
        flag = True
        output[i] = (word, 'JJ', 'O')
    if flag and label == 'Entity':
        output[i] = (word, 'NNS', 'Entity')
        flag = False

# if 'be' is followed by a relation then delete it
for i in range(len(output) - 1):
    if output[i][0] in ['be','being','been'] and output[i+1][2] == 'Relation':
        output[i] = (output[i][0], 'VB', 'O')
# print(output)
array_list = output
#code starting to update attribute

# to read the existing domain_dict from attributes.py
with open("attributes.py", "r") as f:
    attributes_contents = f.read()

# Convert the domain_dict string to a dictionary
attributes_dict = ast.literal_eval(attributes_contents.split("=")[1].strip())
def synonym_antonym_extractor(phrase):
    synonyms = []
    # Extract synonyms for the given phrase
    for syn in wn.synsets(phrase):
        for l in syn.lemmas():
            synonyms.append(l.name())
    synonyms = set(synonyms)
    # Find common keys (entity names) between synonyms and domain_dict
    common_keys = synonyms.intersection(domain_dict.keys())
    return common_keys

# Initialize variables
entity_ids = {}
entities = []
relations = []
entity_counter = 1
relation_counter = 1
new_attributes_dict = {}

# Create the entity_ids dictionary
for i, (word, pos, label) in enumerate(array_list):
    if label == 'Entity' or label == 'Relation':
        entity_ids.setdefault(word, f"ent_{len(entity_ids) + 1}")

for i, (word, pos, label) in enumerate(array_list):
    if label == 'Entity':
        # Create entity dictionary
        entity_dict = {
            "id": entity_ids[word],
            "name": word,
            "POS": pos,
            "properties": [
                {
                    "attribute": []
                }
            ]
        }
        entities.append(entity_dict)
    elif label == 'Relation':
        relation_name = word
        if len(entities) >= 1:
            relation_entity_ids = []
            relation_entity_pos = []
            # Find the entity names preceding and succeeding the relation
            for j in range(i-1, -1, -1):
                if array_list[j][2] == 'Entity':
                    entity_name = array_list[j][0]
                    if entity_name in entity_ids:
                        relation_entity_ids.append(entity_ids[entity_name])
                        relation_entity_pos.append(array_list[j][1])
                        break
            for j in range(i+1, len(array_list)):
                if array_list[j][2] == 'Entity':
                    entity_name = array_list[j][0]
                    if entity_name in entity_ids:
                        relation_entity_ids.append(entity_ids[entity_name])
                        relation_entity_pos.append(array_list[j][1])
                        break
            # Create relation dictionary
            if len(relation_entity_ids) == 2:
                relations.append({
                    "id": f"r{relation_counter}",
                    "name": relation_name,
                    "entity_ids": [
                        {
                            "id": relation_entity_ids[0],
                            "pos": relation_entity_pos[0]
                        },
                        {
                            "id": relation_entity_ids[1],
                            "pos": relation_entity_pos[1]
                        }
                    ]
                })
                relation_counter += 1

# Create the ER diagram dictionary
er_diagram = {
    "entities": entities,
    "relations": relations
}

# Convert the ER diagram dictionary to JSON string
json_ = json.dumps(er_diagram, indent=4)
# print(json_)
# Remove entity duplicates
entities_dict = {}  # Dictionary to store unique entities

json_data = json.loads(json_)  # Convert JSON string to dictionary

# Iterate over entities
for entity in json_data['entities']:
    entity_id = entity['id']
    if entity_id in entities_dict:
        # Entity already exists, update its properties
        existing_entity = entities_dict[entity_id]
        existing_properties = existing_entity['properties']
        new_properties = entity['properties']
        for new_prop in new_properties:
            new_attr = new_prop['attribute']
            if new_attr not in [prop['attribute'] for prop in existing_properties]:
                existing_properties.append(new_prop)
    else:
        # Add entity to dictionary
        entities_dict[entity_id] = entity

stored_relations = {}  # Dictionary to store unique relations
i = 0

while i < len(relations):
    relation = relations[i]
    duplicate_found = False
    highest_degree_pos = {}

    if relation['name'] in stored_relations:
        duplicate_relation = stored_relations[relation['name']]
        if set([entity['id'] for entity in duplicate_relation['entity_ids']]) == set(
                [entity['id'] for entity in relation['entity_ids']]):
            duplicate_found = True
    else:
        stored_relations[relation['name']] = relation

    if not duplicate_found:
        for other_relation in relations:
            if other_relation != relation and other_relation['name'] == relation['name']:
                if set([entity['id'] for entity in other_relation['entity_ids']]) == set(
                        [entity['id'] for entity in relation['entity_ids']]) or \
                   set([entity['id'] for entity in other_relation['entity_ids']]) == set(
                        [entity['id'] for entity in relation['entity_ids'][::-1]]):
                    for entity in relation['entity_ids']:
                        entity_id = entity['id']
                        pos = entity['pos']
                        if entity_id in highest_degree_pos:
                            if pos == 'NNS' and highest_degree_pos[entity_id] == 'NN':
                                highest_degree_pos[entity_id] = 'NNS'
                        else:
                            highest_degree_pos[entity_id] = pos

                        other_entity = next(
                            (other_entity for other_entity in other_relation['entity_ids'] if
                             other_entity['id'] == entity_id), None)
                        if other_entity:
                            other_pos = other_entity['pos']
                            if other_pos != pos:
                                if other_pos == 'NNS' and pos == 'NN':
                                    highest_degree_pos[entity_id] = 'NNS'

                    duplicate_found = True
                    break

        for entity in relation['entity_ids']:
            entity_id = entity['id']
            pos = entity['pos']
            highest_degree = highest_degree_pos.get(entity_id)
            if highest_degree and highest_degree != pos:
                entity['pos'] = highest_degree

        i += 1
    else:
        relations.pop(i)

seen_relations = set()  # Set to keep track of seen relations
i = 0
while i < len(relations):
    relation = relations[i]
    # Check if the relation has been seen before
    if (relation['name'], tuple(sorted(e['id'] for e in relation['entity_ids']))) in seen_relations:
        relations.pop(i)
    else:
        seen_relations.add((relation['name'], tuple(sorted(e['id'] for e in relation['entity_ids']))))
        i += 1

# Convert dictionary back to list of entities
entities_list = list(entities_dict.values())

# Create the ER diagram dictionary
er_diagram = {
    "Entities": entities_list,
    "Relationships": relations
}

# Convert list of entities back to JSON string
output_json = json.dumps(er_diagram, indent=4)

# print(output_json)
entities_final = []  # List to store final entities
attributes = {}  # Dictionary to store attributes

# Extract attributes from existing entities
for entity in er_diagram["Entities"]:
    word = entity["name"]
    entity_attributes = [attr["attribute"] for attr in entity["properties"]]
    attributes[word] = entity_attributes

# Process entities
for word, entity_attributes in attributes.items():
    if word in domain_dict.keys():
        # Entity attributes found in the domain_dict
        entity_final = {
            "id": entity_ids[word],
            "name": word,
            "POS": pos,
            "properties": [
                {
                    "attribute": attribute_name
                } for attribute_name in dict.fromkeys(domain_dict[word])
            ]
        }
    elif len(synonym_antonym_extractor(phrase=word)) != 0:
        # Matching synonyms found in the domain_dict
        matched_word = synonym_antonym_extractor(phrase=word)
        for value in matched_word:
            matched_attr = domain_dict[value]
            user_input = input("We found these similar attributes: " + ", ".join(matched_attr) +
                               " for Entity Name: " + word + ". Do you want to add this (Yes/No):")
            new_attribute = matched_attr
            if user_input in ["yes", "Yes"]:
                # Add matched attributes to entity
                entity_final = {
                    "id": entity_ids[word],
                    "name": word,
                    "POS": pos,
                    "properties": [
                        {
                            "attribute": attribute_name
                        } for attribute_name in dict.fromkeys(domain_dict[value])
                    ]
                }
                # Update domain_dict with new attributes
                if new_attribute:
                    new_attributes_dict[word] = set(new_attribute)
                    attributes_dict.update(new_attributes_dict)
                    # Convert the updated domain_dict back to a string
                    new_attributes_contents = f"domain_dict = {attributes_dict}"
                    # Write the updated domain_dict to attributes.py
                    with open("attributes.py", "w") as f:
                        f.write(new_attributes_contents)
                break
            else:
                input_string = input("Unfortunately we didn't find attributes for Entity Name: " + word +
                                     ", Please Enter attributes separated with space:")
                new_attribute = input_string.split()
                # Add user-defined attributes to entity
                entity_final = {
                    "id": entity_ids[word],
                    "name": word,
                    "POS": pos,
                    "properties": [
                        {
                            "attribute": attribute
                        } for attribute in new_attribute
                    ]
                }
                # Update domain_dict with new attributes
                if new_attribute:
                    new_attributes_dict[word] = set(new_attribute)
                    attributes_dict.update(new_attributes_dict)
                    # Convert the updated domain_dict back to a string
                    new_attributes_contents = f"domain_dict = {attributes_dict}"
                    # Write the updated domain_dict to attributes.py
                    with open("attributes.py", "w") as f:
                        f.write(new_attributes_contents)
                break
    else:
        # No attributes found, prompt user for input
        input_string = input("Unfortunately we didn't find attributes for Entity Name: " + word +
                             ", Please Enter attributes separated with space:")
        new_attribute = input_string.split()
        # Add user-defined attributes to entity
        entity_final = {
            "id": entity_ids[word],
            "name": word,
            "POS": pos,
            "properties": [
                {
                    "attribute": attribute
                } for attribute in new_attribute
            ]
        }
        # Update domain_dict with new attributes
        if new_attribute:
            new_attributes_dict[word] = set(new_attribute)
            attributes_dict.update(new_attributes_dict)
            # Convert the updated domain_dict back to a string
            new_attributes_contents = f"domain_dict = {attributes_dict}"
            # Write the updated domain_dict to attributes.py
            with open("attributes.py", "w") as f:
                f.write(new_attributes_contents)
    entities_final.append(entity_final)

er_diagram = {
    "entities": entities_final,
    "relations": relations
}

# Print the ER diagram as JSON
json_final = json.dumps(er_diagram, indent=4)
print(json_final)
# alligned graph

# Load the ER diagram JSON data
er_diagram = json.loads(json_final)

# Create a Graphviz graph with left-to-right direction
graph = Digraph(graph_attr={'rankdir': 'LR'})

# Add the entities and their properties as nodes to the graph
for entity in er_diagram['entities']:
    entity_id = entity['id']
    graph.node(entity_id, entity['name'], shape='rectangle', style='filled', fillcolor='lightblue')

    for property in entity['properties']:
        node_id = f"{entity_id}_{property['attribute']}"
        graph.node(node_id, property['attribute'], shape='oval')
        graph.edge(entity_id, node_id, dir='none')

# Add the relations and their entities as nodes to the graph
for relation in er_diagram['relations']:
    relation_id = relation['id']
    graph.node(relation_id, relation['name'], shape='diamond', style='filled', fillcolor='lightgrey')

    entity_ids = relation['entity_ids']
    from_id, to_id = entity_ids[0]['id'], entity_ids[1]['id']
    l1 = '1' if entity_ids[0]['pos'] == 'NN' else 'm'
    l2 = '1' if entity_ids[1]['pos'] == 'NN' else 'm'
#     l2 = '1' if entity_ids[0]['pos'] in ['NNS','NNP'] else 'm'
    graph.edge(from_id, relation_id, dir='none',label = l1)
    graph.edge(relation_id, to_id, dir='none',label = l2)
    
# Render the Graphviz graph and save it as a PNG image
graph.render('er_diagram', format='png')

# Open the image using PIL library
im = Image.open('er_diagram.png')

# Display the image in the console
im.show()
# Function to update the dataset with a new word
def update_dataset(word, pos, label, df):
    # Check if the word and pos already exist in the dataset
    if df[(df['Word'] == word) & (df['POS'] == pos)].empty:
        # If not, create a new dataframe with the new row
        new_row = pd.DataFrame({'Word': [word], 'POS': [pos], 'label': [label]})
        df = pd.concat([df, new_row], ignore_index=True)
    else:
        # If the word and pos already exist in the dataset, update the label
        df.loc[(df['Word'] == word) & (df['POS'] == pos), 'label'] = label
    # Write the updated dataframe to the 'learning_dataset.csv' file
    df.to_csv('learning_dataset.csv', index=False)
    # Return the updated dataframe
    return df
#interactive learning

# Iterate over the list of tuples
feedback = input("How is the ERD(good/bad): ")

if feedback == "good":
    for word, pos, label in array_list:
        # Call the update_dataset function for every word
#         print(word,pos,label)
        df = pd.read_csv("learning_dataset.csv")
        update_dataset(word, pos, label, df)
    print("I sincerely appreciate your feedback and will use it to enhance our performance. Thank you!")
elif feedback == "bad":
    for word, pos, label in array_list:
        # Call the update_dataset function for every word
#         print(word, pos, label)
        df = pd.read_csv("learning_dataset.csv")
        update_dataset(word, pos, label, df)
    # Take multiple words, their POS, and updated labels as input from the user
    words = input("Enter words separated by space: ").split(' ')
    poss = input("Enter POS tags separated by space: ").split(' ')
    labels = input("Enter labels separated by space: ").split(' ')
    for i in range(len(words)):
        df = pd.read_csv("learning_dataset.csv")
        if ((df['Word'] == words[i]) & (df['POS'] == poss[i])).any():
            update_dataset(words[i], poss[i], labels[i], df)
        else:
            print("I'm sorry, but the words you mentioned do not appear in the sentence. Could you please double-check?")
    print("Thank you for your valuable feedback. It helps me improve.")
else:
    print("Please provide your feedback for me to improve.")

        